# Get data and packages

In [1]:
# %%capture
# !pip install vaex-core vaex-hdf5
# !pip install -U numpy

In [2]:
# !aws s3 cp s3://xdss-public-datasets/demos/taxi_1B.hdf5 ../datasets/taxi_1B.hdf5

## Prep benchmarks

In [3]:
import gc
from src.benchmarks_utils import benchmark, get_results
from src.vaex_utils import *
from src.config import repetitions

name = 'vaex'
# data_path = '../datasets/taxi_1B.hdf5'
data_path = '/data/yellow_taxi_2009_2015_f32.hdf5'
results_path = f"../results/{name}_1b.csv"
benchmarks = {}
print(f"test for {repetitions} repetitions")

test for 1 repetitions


# Benchmark

In [4]:
# Load data
data = read_file(data_path=data_path)
data['pickup_hour'] = data.pickup_datetime.dt.hour
print(f"size: {data.shape[0]} with {data.shape[1]} columns")

size: 1173057927 with 19 columns


In [5]:
data

#,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,payment_type,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_hour
0,VTS,2009-01-04 02:52:00.000000000,2009-01-04 03:02:00.000000000,1,CASH,2.630000114440918,-73.99195861816406,40.72156524658203,nan,nan,-73.99380493164062,40.6959228515625,8.899999618530273,0.5,nan,0.0,0.0,9.399999618530273,2
1,VTS,2009-01-04 03:31:00.000000000,2009-01-04 03:38:00.000000000,3,Credit,4.550000190734863,-73.98210144042969,40.736289978027344,nan,nan,-73.95584869384766,40.768028259277344,12.100000381469727,0.5,nan,2.0,0.0,14.600000381469727,3
2,VTS,2009-01-03 15:43:00.000000000,2009-01-03 15:57:00.000000000,5,Credit,10.350000381469727,-74.0025863647461,40.73974609375,nan,nan,-73.86997985839844,40.770225524902344,23.700000762939453,0.0,nan,4.739999771118164,0.0,28.440000534057617,15
3,DDS,2009-01-01 20:52:58.000000000,2009-01-01 21:14:00.000000000,1,CREDIT,5.0,-73.9742660522461,40.79095458984375,nan,nan,-73.9965591430664,40.731849670410156,14.899999618530273,0.5,nan,3.049999952316284,0.0,18.450000762939453,20
4,DDS,2009-01-24 16:18:23.000000000,2009-01-24 16:24:56.000000000,1,CASH,0.4000000059604645,-74.00157928466797,40.719383239746094,nan,nan,-74.00837707519531,40.7203483581543,3.700000047683716,0.0,nan,0.0,0.0,3.700000047683716,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"1,173,057,922",VTS,2015-12-31 23:59:56.000000000,2016-01-01 00:08:18.000000000,5,1,1.2000000476837158,-73.99381256103516,40.72087097167969,1.0,0.0,-73.98621368408203,40.722469329833984,7.5,0.5,0.5,1.7599999904632568,0.0,10.5600004196167,23
"1,173,057,923",CMT,2015-12-31 23:59:58.000000000,2016-01-01 00:05:19.000000000,2,2,2.0,-73.96527099609375,40.76028060913086,1.0,0.0,-73.93951416015625,40.75238800048828,7.5,0.5,0.5,0.0,0.0,8.800000190734863,23
"1,173,057,924",CMT,2015-12-31 23:59:59.000000000,2016-01-01 00:12:55.000000000,2,2,3.799999952316284,-73.98729705810547,40.739078521728516,1.0,0.0,-73.9886703491211,40.69329833984375,13.5,0.5,0.5,0.0,0.0,14.800000190734863,23
"1,173,057,925",VTS,2015-12-31 23:59:59.000000000,2016-01-01 00:10:26.000000000,1,2,1.9600000381469727,-73.99755859375,40.72569274902344,1.0,0.0,-74.01712036132812,40.705322265625,8.5,0.5,0.5,0.0,0.0,9.800000190734863,23


In [7]:
# benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
# benchmarks['length'] = benchmark(length, data, repetitions=repetitions)
# benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
# benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
# benchmarks['mean of sum']= benchmark(mean_of_sum, data, repetitions=repetitions)
# benchmarks['mean of columns']= benchmark(mean_of_product, data, repetitions=repetitions)
# benchmarks['mean of complicated arithmetic operation']= benchmark(mean_of_complicated_arithmetic_operation, data, repetitions=repetitions)
# benchmarks['value counts']= benchmark(value_counts, data, repetitions=repetitions)
# benchmarks['groupby statistics']= benchmark(groupby_statistics, data, repetitions=repetitions)
# # benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)
# print(f"cleaned {gc.collect()} mb")
# benchmarks['join'] = benchmark(join, data, repetitions=repetitions, other=groupby_statistics(data))
# print(f"Done benchmarks on all data")

# filtered
filterd = filter_data(data)
del data

print(f"Prepare filtered data and deleted {gc.collect()} MB")
benchmarks['filtered length'] = benchmark(length, filterd, repetitions=repetitions)
benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
benchmarks['filtered mean of sum'] = benchmark(mean_of_sum , filterd, repetitions=repetitions)
benchmarks['filtered mean of product'] = benchmark(mean_of_product , filterd, repetitions=repetitions)
benchmarks['filtered complicated arithmetic operation'] = benchmark(mean_of_complicated_arithmetic_operation, filterd, repetitions=repetitions)
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))
print(f"Done benchmarks on filterd data")


Prepare filtered data and deleted 20 MB
Done benchmarks on filterd data


In [ ]:
results = get_results(benchmarks, name)
results.to_csv(results_path)
results#.head()

In [ ]:
total = results[name].sum()
print(f'{int(total / 60)} min {total % 60:.1f} seconds')

In [ ]:
res

In [ ]:
!aws s3 cp  ../results/vaex_1b.csv s3://vaex-sagemaker-demo/benchmarks/vaex_1b_results.csv 